In [3]:
import pandas as pd

In [2]:
# Carga de DataFrames desde los archivos parquet
data_games = pd.read_parquet("Data/games_file.parquet")
data_items = pd.read_parquet("Data/items_file.parquet")
data_reviews = pd.read_parquet("Data/reviews_file.parquet")

NameError: name 'pd' is not defined

In [28]:
data_games

,genres,app_name,title,release_date,tags,price,item_id,developer,release_year
0,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140,Kotoshiro,2018.0
1,Free to Play,Ironbound,Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",0.00,643980,Secret Level SRL,2018.0
2,Casual,Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",0.00,670290,Poolians.com,2017.0
3,Action,弹炸人2222,弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",0.99,767400,彼岸领域,2017.0
4,None,Log Challenge,None,None,"['Action', 'Indie', 'Casual', 'Sports']",2.99,773570,None,NaN
...,...,...,...,...,...,...,...,...,...
32130,Casual,Colony On Mars,Colony On Mars,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",1.99,773640,"Nikita ""Ghost_RUS""",2018.0
32131,Casual,LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"['Strategy', 'Indie', 'Casual']",4.99,733530,Sacada,2018.0
32132,Indie,Russian Roads,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.99,610660,Laush Dmitriy Sergeevich,2018.0
32133,Casual,EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",4.99,658870,"xropi,stev3ns",2017.0


Seleccion de columnas relevantes para nuestro merge

In [32]:
# Columnas de items
data_items = data_items[['user_id','item_id','playtime_forever']]

# Agrupar un id de 'user_items'
data_items['item_id'] = data_items['item_id'].astype(str)
data_items['id'] = data_items['user_id'] + data_items['item_id']

# Cambiar el tipo de dato de la columna 'item_id' de 'games'
data_games['item_id'] = data_games['item_id'].astype(str)
 
# Agrupar un id en 'user_reviews'
data_reviews['item_id'] = data_reviews['item_id'].astype(str)
data_reviews['id'] = data_reviews['user_id'] + data_reviews['item_id']

Merge de toda la data

In [1]:
# Merge de la data 'item_id' en reviews y games  
merged_df = data_reviews.merge(data_games, on='item_id', how='left')
# Eliminacion de nulos 
merged_df.dropna(inplace=True)

# generar dataframe definitivo uniendo 'items' con 'merge_df' por el identificador unico 'id' de nombre 'steam'
all_data = data_items.merge(merged_df, on='id')

# renombrando columnas del dataframe 'steam'
all_data = all_data.rename(columns={'user_id_x':'user_id'})
all_data = all_data.rename(columns={'item_id_x':'item_id'})

# eliminando columnas redundantes del dataframe 'steam'
all_data.drop('user_id_y', axis='columns', inplace=True)
all_data.drop('item_id_y', axis='columns', inplace=True)

# cambio de tipos de datos de algunas columnas
all_data['release_date'] = all_data['release_date'].astype(str)
all_data['year'] = all_data['year'].astype(str)
all_data['playtime_forever'] = all_data['playtime_forever'].astype(int)

# seleccionando columnas definitivas para el analisis
all_data = all_data[['id','user_id','item_id','title','genres','developer','release_date','release_year','price','recommend','year','sentiment_analysis','playtime_forever']]

# exportando archivo separado por comas de nombre 'data_steam'
all_data.to_csv('Data/all_data_merge.csv', index=False)

NameError: name 'data_reviews' is not defined

In [ ]:
all_data

,id,user_id,item_id,title,genres,developer,release_date,price,recommend,year,sentiment_analysis,playtime_forever
0,7656119797098247922200,76561197970982479,22200,Zeno Clash,Action,ACE Team,2009-04-21,9.99,True,2011,2,271
1,765611979709824791250,76561197970982479,1250,Killing Floor,Action,Tripwire Interactive,2009-05-14,19.99,True,2011,1,10006
2,js41637227300,js41637,227300,Euro Truck Simulator 2,Indie,SCS Software,2013-01-16,19.99,True,2013,1,551
3,js41637239030,js41637,239030,"Papers, Please",Adventure,3909,2013-08-08,9.99,True,2013,1,349
4,evcentric107200,evcentric,107200,Space Pirates and Zombies,Action,MinMax Games Ltd.,2011-08-15,9.99,True,2014,1,3225
...,...,...,...,...,...,...,...,...,...,...,...,...
32539,76561198239215706730,76561198239215706,730,Counter-Strike: Global Offensive,Action,Valve,2012-08-21,14.99,True,2015,1,4659
32540,wayfeng730,wayfeng,730,Counter-Strike: Global Offensive,Action,Valve,2012-08-21,14.99,True,2015,1,42740
32541,76561198251004808253980,76561198251004808,253980,Enclave,RPG,Starbreeze,2003-03-10,4.99,True,2015,2,1098
32542,72947282842730,72947282842,730,Counter-Strike: Global Offensive,Action,Valve,2012-08-21,14.99,True,2015,0,33
